In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
from scipy.spatial.distance import mahalanobis as mah_dist # u, v, cov_mat
import os
from tqdm import tqdm
from functools import reduce
import argparse

def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["n_cl"] = len(node_to_connected_clusters[i])
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
1

1

In [5]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
# ramps = [0.02, 0.5]
repeat_num = 1000

In [6]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [7]:
A = np.array(nx.adjacency_matrix(g).todense(), dtype = np.float64)
deg_array = np.array(list(dict(g.degree).values()))

D_inv_A = np.zeros_like(A)

for i in range(num_nodes):
    D_inv_A[i] = A[i] / deg_array[i]


In [8]:
# multi_hop_A = np.linalg.matrix_power(D_inv_A, 2)
# torch.save(multi_hop_A, "A_2hop.pkl")
multi_hop_A = torch.load("A_2hop.pkl")

In [9]:
# set diagonal of 2-hop adjacency to 0
for i in range(num_nodes):
    multi_hop_A[i, i] = 0

In [10]:
node_list = list(g.nodes.keys())

In [11]:
def po_2hop_linear_model(graph, z_vec, alpha=1, beta=1, sigma=0.1, gamma=1, r1=1, r2=0.5):        
    y_vec = alpha + beta * z_vec + sigma * np.random.normal(size=(num_nodes, 1)) + gamma * (
    r1 * np.matmul(D_inv_A, z_vec) + r2 * np.matmul(multi_hop_A, z_vec)
)
    
    for i in range(num_nodes):
        graph.nodes[node_list[i]]["y"] = y_vec[i][0]
        graph.nodes[node_list[i]]["z"] = z_vec[i][0]
        # nx.set_node_attributes(graph, y_vec, "y")

In [ ]:
# cluster-level cr

repeat_num = 1000

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):
# for seed in tqdm(range(151, repeat_num)): # interupted at 151
    
    np.random.seed(seed)     
    # rollout_index = np.random.uniform(0, 1, size=(num_cluster))   
    rollout_index = np.random.uniform(0, 1, size=(num_nodes))   
    
    for num_step in range(len(ramps)): 
        torch.manual_seed(2)
        # p_list = ramps[num_step:]
        p_list = [0, ramps[num_step]]
    
        print(f"Seed: {seed}")
        
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            # z_vector = np.zeros((num_nodes)) 
            # tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            # tr_units = list(reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters]))
            # z_vector[tr_units] = 1
        
            z_vector = (rollout_index <= np.quantile(rollout_index, p)).astype(int) # unit-level
            po_2hop_linear_model(g, z_vector.reshape(-1,1)) # add dimension
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
    
    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/incre_2hop_CR_unit.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1171
Epoch 0, Treat Prop: 0.02, Loss: 2.3677
Epoch 50, Treat Prop: 0.00, Loss: 0.1631
Epoch 50, Treat Prop: 0.02, Loss: 0.1810
Epoch 100, Treat Prop: 0.00, Loss: 0.0259
Epoch 100, Treat Prop: 0.02, Loss: 0.0407
Epoch 150, Treat Prop: 0.00, Loss: 0.0154
Epoch 150, Treat Prop: 0.02, Loss: 0.0240
Epoch 200, Treat Prop: 0.00, Loss: 0.0138
Epoch 200, Treat Prop: 0.02, Loss: 0.0196
Epoch 250, Treat Prop: 0.00, Loss: 0.0130
Epoch 250, Treat Prop: 0.02, Loss: 0.0173
Epoch 300, Treat Prop: 0.00, Loss: 0.0125
Epoch 300, Treat Prop: 0.02, Loss: 0.0161
Epoch 350, Treat Prop: 0.00, Loss: 0.0122
Epoch 350, Treat Prop: 0.02, Loss: 0.0154
0.015079322271049023
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1199
Epoch 0, Treat Prop: 0.05, Loss: 2.3848
Epoch 50, Treat Prop: 0.00, Loss: 0.1506
Epoch 50, Treat Prop: 0.05, Loss: 0.2266
Epoch 100, Treat Prop: 0.00, Loss: 0.0305
Epoch 100, Treat Prop: 0.05, Loss: 0.0688
Epoch 150, Treat Prop: 0.00, Loss: 0.0189
Epoch 150

  0%|          | 1/1000 [00:17<4:43:27, 17.02s/it]

0.08090205490589142
Seed: 1
Epoch 0, Treat Prop: 0.00, Loss: 8.1039
Epoch 0, Treat Prop: 0.02, Loss: 2.3816
Epoch 50, Treat Prop: 0.00, Loss: 0.1624
Epoch 50, Treat Prop: 0.02, Loss: 0.1826
Epoch 100, Treat Prop: 0.00, Loss: 0.0256
Epoch 100, Treat Prop: 0.02, Loss: 0.0419
Epoch 150, Treat Prop: 0.00, Loss: 0.0151
Epoch 150, Treat Prop: 0.02, Loss: 0.0235
Epoch 200, Treat Prop: 0.00, Loss: 0.0137
Epoch 200, Treat Prop: 0.02, Loss: 0.0192
Epoch 250, Treat Prop: 0.00, Loss: 0.0130
Epoch 250, Treat Prop: 0.02, Loss: 0.0170
Epoch 300, Treat Prop: 0.00, Loss: 0.0125
Epoch 300, Treat Prop: 0.02, Loss: 0.0158
Epoch 350, Treat Prop: 0.00, Loss: 0.0122
Epoch 350, Treat Prop: 0.02, Loss: 0.0151
0.014783879742026329
Seed: 1
Epoch 0, Treat Prop: 0.00, Loss: 8.1153
Epoch 0, Treat Prop: 0.05, Loss: 2.3797
Epoch 50, Treat Prop: 0.00, Loss: 0.1512
Epoch 50, Treat Prop: 0.05, Loss: 0.2289
Epoch 100, Treat Prop: 0.00, Loss: 0.0297
Epoch 100, Treat Prop: 0.05, Loss: 0.0696
Epoch 150, Treat Prop: 0.00, Lo

  0%|          | 2/1000 [00:33<4:42:37, 16.99s/it]

...

0.07777942717075348
Seed: 147
Epoch 0, Treat Prop: 0.00, Loss: 8.1191
Epoch 0, Treat Prop: 0.02, Loss: 2.3590
Epoch 50, Treat Prop: 0.00, Loss: 0.1604
Epoch 50, Treat Prop: 0.02, Loss: 0.1820
Epoch 100, Treat Prop: 0.00, Loss: 0.0264
Epoch 100, Treat Prop: 0.02, Loss: 0.0419
Epoch 150, Treat Prop: 0.00, Loss: 0.0156
Epoch 150, Treat Prop: 0.02, Loss: 0.0242
Epoch 200, Treat Prop: 0.00, Loss: 0.0140
Epoch 200, Treat Prop: 0.02, Loss: 0.0200
Epoch 250, Treat Prop: 0.00, Loss: 0.0132
Epoch 250, Treat Prop: 0.02, Loss: 0.0178
Epoch 300, Treat Prop: 0.00, Loss: 0.0127
Epoch 300, Treat Prop: 0.02, Loss: 0.0166
Epoch 350, Treat Prop: 0.00, Loss: 0.0123
Epoch 350, Treat Prop: 0.02, Loss: 0.0159
0.01555159967392683
Seed: 147
Epoch 0, Treat Prop: 0.00, Loss: 8.1146
Epoch 0, Treat Prop: 0.05, Loss: 2.3689
Epoch 50, Treat Prop: 0.00, Loss: 0.1506
Epoch 50, Treat Prop: 0.05, Loss: 0.2226
Epoch 100, Treat Prop: 0.00, Loss: 0.0308
Epoch 100, Treat Prop: 0.05, Loss: 0.0698
Epoch 150, Treat Prop: 0.00,

 15%|█▍        | 148/1000 [41:43<3:51:46, 16.32s/it]

0.07891225069761276
Seed: 148
Epoch 0, Treat Prop: 0.00, Loss: 8.1258
Epoch 0, Treat Prop: 0.02, Loss: 2.3706
Epoch 50, Treat Prop: 0.00, Loss: 0.1620
Epoch 50, Treat Prop: 0.02, Loss: 0.1817
Epoch 100, Treat Prop: 0.00, Loss: 0.0256
Epoch 100, Treat Prop: 0.02, Loss: 0.0410
Epoch 150, Treat Prop: 0.00, Loss: 0.0152
Epoch 150, Treat Prop: 0.02, Loss: 0.0235
Epoch 200, Treat Prop: 0.00, Loss: 0.0137
Epoch 200, Treat Prop: 0.02, Loss: 0.0192
Epoch 250, Treat Prop: 0.00, Loss: 0.0130
Epoch 250, Treat Prop: 0.02, Loss: 0.0170
Epoch 300, Treat Prop: 0.00, Loss: 0.0125
Epoch 300, Treat Prop: 0.02, Loss: 0.0157
Epoch 350, Treat Prop: 0.00, Loss: 0.0122
Epoch 350, Treat Prop: 0.02, Loss: 0.0150
0.014656604267656803
Seed: 148
Epoch 0, Treat Prop: 0.00, Loss: 8.1054
Epoch 0, Treat Prop: 0.05, Loss: 2.3900
Epoch 50, Treat Prop: 0.00, Loss: 0.1516
Epoch 50, Treat Prop: 0.05, Loss: 0.2256
Epoch 100, Treat Prop: 0.00, Loss: 0.0290
Epoch 100, Treat Prop: 0.05, Loss: 0.0670
Epoch 150, Treat Prop: 0.00

 15%|█▍        | 149/1000 [41:59<3:51:11, 16.30s/it]

0.07975839823484421
Seed: 149
Epoch 0, Treat Prop: 0.00, Loss: 8.1154
Epoch 0, Treat Prop: 0.02, Loss: 2.3773
Epoch 50, Treat Prop: 0.00, Loss: 0.1630
Epoch 50, Treat Prop: 0.02, Loss: 0.1825
Epoch 100, Treat Prop: 0.00, Loss: 0.0258
Epoch 100, Treat Prop: 0.02, Loss: 0.0411
Epoch 150, Treat Prop: 0.00, Loss: 0.0156
Epoch 150, Treat Prop: 0.02, Loss: 0.0237
Epoch 200, Treat Prop: 0.00, Loss: 0.0140
Epoch 200, Treat Prop: 0.02, Loss: 0.0194
Epoch 250, Treat Prop: 0.00, Loss: 0.0133
Epoch 250, Treat Prop: 0.02, Loss: 0.0172
Epoch 300, Treat Prop: 0.00, Loss: 0.0128
Epoch 300, Treat Prop: 0.02, Loss: 0.0160
Epoch 350, Treat Prop: 0.00, Loss: 0.0125
Epoch 350, Treat Prop: 0.02, Loss: 0.0153
0.014901774935424328
Seed: 149
Epoch 0, Treat Prop: 0.00, Loss: 8.1204
Epoch 0, Treat Prop: 0.05, Loss: 2.3874
Epoch 50, Treat Prop: 0.00, Loss: 0.1514
Epoch 50, Treat Prop: 0.05, Loss: 0.2283
Epoch 100, Treat Prop: 0.00, Loss: 0.0298
Epoch 100, Treat Prop: 0.05, Loss: 0.0691
Epoch 150, Treat Prop: 0.00